In [3]:
%run model_functions.ipynb

In [4]:
datasets = []
for genre in ['academic', 'fiction', 'newspapers', 'conversation']:
    dataset = pd.read_csv('data/train/{}/words.csv'.format(genre),
                          encoding='ISO--8859-1', na_filter=False)
    display(dataset.head())
    datasets.append(dataset)

,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre
0,25070,HER,she,DPS,,,1267,a6u-fragment02,a6u,academic
1,25071,DRESS,dress,NN1,,,1267,a6u-fragment02,a6u,academic
2,25072,HANGS,hang,VVZ,,,1267,a6u-fragment02,a6u,academic
3,25073,HERE',here',NP0,,,1267,a6u-fragment02,a6u,academic
4,25074,DE-FROCKING,de-frock,VVG,mrw,met,1267,a6u-fragment02,a6u,academic


,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre
0,38263,The,the,AT0,,,1947,ab9-fragment03,ab9,fiction
1,38264,boy,boy,NN1,,,1947,ab9-fragment03,ab9,fiction
2,38265,and,and,CJC,,,1947,ab9-fragment03,ab9,fiction
3,38266,the,the,AT0,,,1947,ab9-fragment03,ab9,fiction
4,38267,dog,dog,NN1,,,1947,ab9-fragment03,ab9,fiction


,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre
0,0,Latest,late,AJS,,,0,a1e-fragment01,a1e,newspapers
1,1,corporate,corporate,AJ0,,,0,a1e-fragment01,a1e,newspapers
2,2,unbundler,unbundler,NN1,,,0,a1e-fragment01,a1e,newspapers
3,3,reveals,reveal,VVZ,mrw,met,0,a1e-fragment01,a1e,newspapers
4,4,laid-back,laid-back,AJ0,,,0,a1e-fragment01,a1e,newspapers


,word_id,word,lemma,word_type,function,seg_type,sentence_id,text_id,text_tag,genre
0,156256,Ours,ours,PNP,,,8721,kb7-fragment10,kb7,conversation
1,156257,is,be,VBZ,,,8721,kb7-fragment10,kb7,conversation
2,156258,actually,actually,AV0,,,8721,kb7-fragment10,kb7,conversation
3,156259,a,a,AT0,,,8721,kb7-fragment10,kb7,conversation
4,156260,new,new,AJ0,,,8721,kb7-fragment10,kb7,conversation


In [5]:
dataset = pd.concat(datasets)
len(dataset)

164303

In [6]:
all_sentences = pd.read_csv('data/raw/sentences.csv', encoding='ISO-8859-1')
display(len(all_sentences))
display(all_sentences.sample(5))
all_sentences.tail(3)

16183

,sentence_id,paragraph_id,sentence
5458,5458,1681,But she had given up all hope of Joss Barnet returning that evening and nothing else in the world mattered.
11807,11807,-1,What have you been doing?
1002,1002,325,"If you do not know what a tree or shrub is, exercise clemency and give it a year's grace to demonstrate its worth before you turn it over to the axeman."
11280,11280,-1,And it 's a I mean you get four hundred people on the dance floor in the Sugar House.
0,0,-1,"Latest corporate unbundler reveals laid-back approach: Roland Franklin, who is leading a 697m pound break-up bid for DRG, talks to Frank Kane"


,sentence_id,paragraph_id,sentence
16180,16180,-1,Some else's sitting at their desk?
16181,16181,-1,Well not you know cleaning so I do n't know what kind of work they do.
16182,16182,-1,Oh well if you 're here that 's all right


In [7]:
# Sentences the training set contains.
sentences = all_sentences.merge(dataset)[['sentence_id', 'paragraph_id', 'sentence']].drop_duplicates()
display(sentences['sentence_id'].nunique())
sentences.sample(5)

13134

,sentence_id,paragraph_id,sentence
148814,13146,-1,Oh yeah.
159051,15066,-1,Have you got ta get a card for Julie as well?
121851,8593,2580,We 're more sophisticated in our approach.
36241,2207,729,I 've found a super husband for Harriet.
4730,225,85,Only the Women's Ministry would be given equal license.


## Build paragraphs

In [8]:
paragraph_sentences = sentences[sentences['paragraph_id'] != -1]
display(len(paragraph_sentences))  # Number of sentences that belong to paragraphs.
paragraph_sentences.sample(5)

6561

,sentence_id,paragraph_id,sentence
21398,1333,419,"Although this is not in itself to be dismissed as a tactic, it has diminished the complexity of Kahlo's achievement as a specifically Mexican painter, operating within the particularities of her historical moment."
5498,347,127,The success of the Ahlbergs' working relationship must have a lot to do with their long and happy marriage.
25651,1583,491,If the person in default fails to comply with an abatement notice the local authority is obliged to make a complaint to the magistrates.
28829,1803,579,"Six months later, the day before Rome's liberation by the Allies, Amaldi was found."
50198,3147,1045,"In describing the project, a non-punitive model for understanding probation practice will be developed."


In [9]:
paragraphs = paragraph_sentences.groupby('paragraph_id')['sentence'].agg(list).apply(lambda x: ' '.join(x))
display(len(paragraphs))
paragraphs.head()

2067

paragraph_id
3    IT SEEMS that Roland Franklin, the latest unbundler to appear in the UK, has made a fatal error in the preparation of his £697m break-up bid for stationery and packaging group DRG. He has not properly investigated the target's dining facilities.                                                                                                                   
4    The 63-year-old head of Pembridge Investments, through which the bid is being mounted says, rule number one in this business is: the more luxurious the luncheon rooms at headquarters, the more inefficient the business. If he had taken his own rule seriously, he would have found out that DRG has a very modest self-service canteen at its Bristol head office.
5    There are other things he has, on his own admission, not fully investigated, like the value of the DRG properties, or which part of the DRG business he would keep after the break up. When the bid was launched last week, Mr Franklin faced some critici

## Save paragraphs

This saves all the paragraphs we created from our sentences.
```python
!rm data/train/paragraphs/*
for i, paragraph in paragraphs.iteritems():
    file_name = "data/train/paragraphs/paragraph_{}.txt".format(i)
    with open(file_name, 'w', encoding='utf-8') as f:
        print('Writing {}'.format(file_name))
        f.write(paragraph)
```

Next, run the following.
```bash
python scripts/paragraph_chain.py
```

## Explore Longest Chains

```python
longest_chains = pd.read_csv('data/train/longest_chains.csv')  # Paragraphs with longest chains != ''

longest_chain_in_paragraphs = longest_chains.merge(paragraphs, left_on='paragraph_id', right_index=True)

# We should have a row for each paragraph with a non-empty longest_chain.
display(len(paragraphs))
display(sum(longest_chains['paragraph_id'].isin(paragraphs.index)))
display(len(longest_chain_in_paragraphs['paragraph_id'].drop_duplicates()))
display(len(longest_chain_in_paragraphs))
display(len(longest_chains))

longest_chain_in_paragraphs.head()
```

In [11]:
len(longest_chains)

1519

Looks pretty good!

## Now let's put our code into reuseable and automated functions!

In [243]:
dataset = pd.read_csv('data/train/academic/words.csv'.format(genre),
                      encoding='ISO--8859-1', na_filter=False)

In [110]:
def build_paragraphs(dataset):
    all_sentences = pd.read_csv('data/raw/sentences.csv', encoding='ISO-8859-1')

    # Sentences the given dataset contains.
    sentences = all_sentences.merge(dataset)[['sentence_id', 'paragraph_id', 'sentence']].drop_duplicates()

    paragraph_sentences = sentences[sentences['paragraph_id'] != -1]

    paragraphs = paragraph_sentences.groupby('paragraph_id')['sentence'].agg(list).apply(lambda x: ' '.join(x))
    return paragraphs


def save_paragraphs(paragraphs):
    try:
        files = glob.glob('data/paragraphs/*')
        for f in files:
            os.remove(f)
    except:
        os.mkdir('data/paragraphs')
        
    for i, paragraph in paragraphs.iteritems():
        file_name = "data/paragraphs/paragraph_{}.txt".format(i)
        with open(file_name, 'w', encoding='utf-8') as f:
            print('Writing {}'.format(file_name))
            f.write(paragraph)  


def get_longest_chains():
    # Go to ELKB directory from the parent directory of this file.
#     dir = os.path.dirname(os.path.realpath(__file__))
#     os.chdir(os.path.join(dir, '../ELKB'))

    with open('data/longest_chains.csv', 'w', encoding='utf-8') as f:
        csv_writer = csv.writer(f, delimiter=',')
        csv_writer.writerow(('paragraph_id', 'longest_chain'))

        for file_name in glob.glob('data/paragraphs/paragraph*.txt'):
            paragraph_id = os.path.splitext(os.path.basename(file_name))[
                0].split('_')[-1]

            with open(file_name, 'r', encoding='utf-8') as g:
                result = subprocess.run(
                    'java applications/LexicalChain -f ../{}'.format(file_name),
                    cwd='ELKB/',
                    stdout=subprocess.PIPE)

                result = result.stdout.decode('utf-8')

                result = result.strip().split('\n')

                # Does not contain a lexical chain.
                # First 3 lines are descriptives by the script.
                if len(result) < 4:
                    continue

                longest_chain = []
                len_longest_chain = 0
                for r in result[3:]:
                    chain = [word.strip()
                             for word in r.split('[')[0].split(',')]
                    if len(chain) >= len_longest_chain:
                        longest_chain.extend(chain)
                        
                    len_longest_chain = len(chain)
                    
                # longest_chain = result[3]
                # longest_chain = [word.strip() for word in longest_chain.split('[')[0].split(',')]
                longest_chain_str = ' '.join(longest_chain)

                # line = '{},{}'.format(paragraph_id, longest_chain)
                # f.write(line + '\n')
                row = [paragraph_id, longest_chain_str]
#                 print(row)
                print('Writing longest chain for {} with {} words!\n'
                      .format(file_name, len(longest_chain)))
                csv_writer.writerow(row)

In [111]:
def get_lc_feature(dataset):
    longest_chains = pd.read_csv('data/longest_chains.csv')  # Paragraphs with longest chains != ''

    longest_chain_in_paragraphs = longest_chains.merge(paragraphs, left_on='paragraph_id', right_index=True)

    words_and_longest_chains = (
        longest_chain_in_paragraphs
        .merge(sentences)
        .merge(dataset[['sentence_id', 'word_id', 'word']])
    )

    words_and_longest_chains['longest_chain_list'] = (
        words_and_longest_chains['longest_chain']
        .apply(lambda x: set(x.split(' ')))
    )

    words_and_longest_chains['is_in_longest_chain'] = [
        word in chain for (word, chain) in zip(
            words_and_longest_chains['word'],
            words_and_longest_chains['longest_chain_list']
        )
    ]

    return words_and_longest_chains[['word_id', 'is_in_longest_chain']]

def add_lc_feature(dataset, genre):
    paragraphs = build_paragraphs(dataset)
    save_paragraphs(paragraphs)
    
    get_longest_chains(genre)
    
    lc_feature = get_lc_feature(dataset)
    
    dataset_with_lc_feature = dataset.merge(lc_feature, how='left')
    dataset_with_lc_feature['is_in_longest_chain'].fillna(False)

    return dataset_with_lc_feature

In [112]:
dataset = pd.read_csv('data/raw/words.csv'.format(genre),
                      encoding='ISO--8859-1', na_filter=False)
paragraphs = build_paragraphs(dataset)
save_paragraphs(paragraphs)
get_longest_chains()

Writing data/paragraphs/paragraph_3.txt
Writing data/paragraphs/paragraph_4.txt
Writing data/paragraphs/paragraph_5.txt
Writing data/paragraphs/paragraph_6.txt
Writing data/paragraphs/paragraph_7.txt
Writing data/paragraphs/paragraph_8.txt
Writing data/paragraphs/paragraph_9.txt
Writing data/paragraphs/paragraph_10.txt
Writing data/paragraphs/paragraph_11.txt
Writing data/paragraphs/paragraph_12.txt
Writing data/paragraphs/paragraph_13.txt
Writing data/paragraphs/paragraph_14.txt
Writing data/paragraphs/paragraph_15.txt
Writing data/paragraphs/paragraph_16.txt
Writing data/paragraphs/paragraph_17.txt
Writing data/paragraphs/paragraph_18.txt
Writing data/paragraphs/paragraph_19.txt
Writing data/paragraphs/paragraph_20.txt
Writing data/paragraphs/paragraph_21.txt
Writing data/paragraphs/paragraph_22.txt
Writing data/paragraphs/paragraph_23.txt
Writing data/paragraphs/paragraph_24.txt
Writing data/paragraphs/paragraph_25.txt
Writing data/paragraphs/paragraph_26.txt
Writing data/paragraphs

Writing data/paragraphs/paragraph_219.txt
Writing data/paragraphs/paragraph_220.txt
Writing data/paragraphs/paragraph_221.txt
Writing data/paragraphs/paragraph_222.txt
Writing data/paragraphs/paragraph_223.txt
Writing data/paragraphs/paragraph_224.txt
Writing data/paragraphs/paragraph_225.txt
Writing data/paragraphs/paragraph_226.txt
Writing data/paragraphs/paragraph_227.txt
Writing data/paragraphs/paragraph_228.txt
Writing data/paragraphs/paragraph_229.txt
Writing data/paragraphs/paragraph_230.txt
Writing data/paragraphs/paragraph_231.txt
Writing data/paragraphs/paragraph_232.txt
Writing data/paragraphs/paragraph_233.txt
Writing data/paragraphs/paragraph_234.txt
Writing data/paragraphs/paragraph_235.txt
Writing data/paragraphs/paragraph_236.txt
Writing data/paragraphs/paragraph_237.txt
Writing data/paragraphs/paragraph_238.txt
Writing data/paragraphs/paragraph_239.txt
Writing data/paragraphs/paragraph_240.txt
Writing data/paragraphs/paragraph_241.txt
Writing data/paragraphs/paragraph_

Writing data/paragraphs/paragraph_424.txt
Writing data/paragraphs/paragraph_425.txt
Writing data/paragraphs/paragraph_426.txt
Writing data/paragraphs/paragraph_427.txt
Writing data/paragraphs/paragraph_428.txt
Writing data/paragraphs/paragraph_429.txt
Writing data/paragraphs/paragraph_430.txt
Writing data/paragraphs/paragraph_431.txt
Writing data/paragraphs/paragraph_432.txt
Writing data/paragraphs/paragraph_433.txt
Writing data/paragraphs/paragraph_434.txt
Writing data/paragraphs/paragraph_435.txt
Writing data/paragraphs/paragraph_436.txt
Writing data/paragraphs/paragraph_437.txt
Writing data/paragraphs/paragraph_438.txt
Writing data/paragraphs/paragraph_439.txt
Writing data/paragraphs/paragraph_440.txt
Writing data/paragraphs/paragraph_441.txt
Writing data/paragraphs/paragraph_442.txt
Writing data/paragraphs/paragraph_443.txt
Writing data/paragraphs/paragraph_444.txt
Writing data/paragraphs/paragraph_445.txt
Writing data/paragraphs/paragraph_446.txt
Writing data/paragraphs/paragraph_

Writing data/paragraphs/paragraph_626.txt
Writing data/paragraphs/paragraph_627.txt
Writing data/paragraphs/paragraph_628.txt
Writing data/paragraphs/paragraph_629.txt
Writing data/paragraphs/paragraph_630.txt
Writing data/paragraphs/paragraph_631.txt
Writing data/paragraphs/paragraph_632.txt
Writing data/paragraphs/paragraph_633.txt
Writing data/paragraphs/paragraph_634.txt
Writing data/paragraphs/paragraph_635.txt
Writing data/paragraphs/paragraph_636.txt
Writing data/paragraphs/paragraph_637.txt
Writing data/paragraphs/paragraph_638.txt
Writing data/paragraphs/paragraph_639.txt
Writing data/paragraphs/paragraph_640.txt
Writing data/paragraphs/paragraph_641.txt
Writing data/paragraphs/paragraph_642.txt
Writing data/paragraphs/paragraph_643.txt
Writing data/paragraphs/paragraph_644.txt
Writing data/paragraphs/paragraph_645.txt
Writing data/paragraphs/paragraph_646.txt
Writing data/paragraphs/paragraph_647.txt
Writing data/paragraphs/paragraph_648.txt
Writing data/paragraphs/paragraph_

Writing data/paragraphs/paragraph_823.txt
Writing data/paragraphs/paragraph_824.txt
Writing data/paragraphs/paragraph_825.txt
Writing data/paragraphs/paragraph_826.txt
Writing data/paragraphs/paragraph_827.txt
Writing data/paragraphs/paragraph_828.txt
Writing data/paragraphs/paragraph_829.txt
Writing data/paragraphs/paragraph_830.txt
Writing data/paragraphs/paragraph_831.txt
Writing data/paragraphs/paragraph_832.txt
Writing data/paragraphs/paragraph_833.txt
Writing data/paragraphs/paragraph_834.txt
Writing data/paragraphs/paragraph_835.txt
Writing data/paragraphs/paragraph_836.txt
Writing data/paragraphs/paragraph_837.txt
Writing data/paragraphs/paragraph_838.txt
Writing data/paragraphs/paragraph_839.txt
Writing data/paragraphs/paragraph_840.txt
Writing data/paragraphs/paragraph_841.txt
Writing data/paragraphs/paragraph_842.txt
Writing data/paragraphs/paragraph_843.txt
Writing data/paragraphs/paragraph_844.txt
Writing data/paragraphs/paragraph_845.txt
Writing data/paragraphs/paragraph_

...

Writing longest chain for data/paragraphs\paragraph_458.txt with 2 words!

Writing longest chain for data/paragraphs\paragraph_46.txt with 5 words!

Writing longest chain for data/paragraphs\paragraph_460.txt with 2 words!

Writing longest chain for data/paragraphs\paragraph_461.txt with 10 words!

Writing longest chain for data/paragraphs\paragraph_462.txt with 2 words!

Writing longest chain for data/paragraphs\paragraph_463.txt with 8 words!

Writing longest chain for data/paragraphs\paragraph_464.txt with 10 words!

Writing longest chain for data/paragraphs\paragraph_465.txt with 12 words!

Writing longest chain for data/paragraphs\paragraph_466.txt with 27 words!

Writing longest chain for data/paragraphs\paragraph_467.txt with 24 words!

Writing longest chain for data/paragraphs\paragraph_468.txt with 7 words!

Writing longest chain for data/paragraphs\paragraph_469.txt with 18 words!

Writing longest chain for data/paragraphs\paragraph_47.txt with 7 words!

Writing longest chain

Writing longest chain for data/paragraphs\paragraph_589.txt with 8 words!

Writing longest chain for data/paragraphs\paragraph_59.txt with 11 words!

Writing longest chain for data/paragraphs\paragraph_590.txt with 14 words!

Writing longest chain for data/paragraphs\paragraph_591.txt with 21 words!

Writing longest chain for data/paragraphs\paragraph_592.txt with 9 words!

Writing longest chain for data/paragraphs\paragraph_593.txt with 6 words!

Writing longest chain for data/paragraphs\paragraph_594.txt with 22 words!

Writing longest chain for data/paragraphs\paragraph_595.txt with 10 words!

Writing longest chain for data/paragraphs\paragraph_596.txt with 5 words!

Writing longest chain for data/paragraphs\paragraph_597.txt with 21 words!

Writing longest chain for data/paragraphs\paragraph_598.txt with 12 words!

Writing longest chain for data/paragraphs\paragraph_599.txt with 13 words!

Writing longest chain for data/paragraphs\paragraph_6.txt with 9 words!

Writing longest chai

Writing longest chain for data/paragraphs\paragraph_701.txt with 2 words!

Writing longest chain for data/paragraphs\paragraph_702.txt with 3 words!

Writing longest chain for data/paragraphs\paragraph_703.txt with 2 words!

Writing longest chain for data/paragraphs\paragraph_705.txt with 2 words!

Writing longest chain for data/paragraphs\paragraph_706.txt with 17 words!

Writing longest chain for data/paragraphs\paragraph_707.txt with 8 words!

Writing longest chain for data/paragraphs\paragraph_708.txt with 11 words!

Writing longest chain for data/paragraphs\paragraph_709.txt with 6 words!

Writing longest chain for data/paragraphs\paragraph_71.txt with 10 words!

Writing longest chain for data/paragraphs\paragraph_710.txt with 4 words!

Writing longest chain for data/paragraphs\paragraph_711.txt with 4 words!

Writing longest chain for data/paragraphs\paragraph_716.txt with 2 words!

Writing longest chain for data/paragraphs\paragraph_717.txt with 2 words!

Writing longest chain f

Writing longest chain for data/paragraphs\paragraph_874.txt with 5 words!

Writing longest chain for data/paragraphs\paragraph_875.txt with 8 words!

Writing longest chain for data/paragraphs\paragraph_876.txt with 5 words!

Writing longest chain for data/paragraphs\paragraph_877.txt with 7 words!

Writing longest chain for data/paragraphs\paragraph_878.txt with 11 words!

Writing longest chain for data/paragraphs\paragraph_879.txt with 12 words!

Writing longest chain for data/paragraphs\paragraph_88.txt with 6 words!

Writing longest chain for data/paragraphs\paragraph_880.txt with 18 words!

Writing longest chain for data/paragraphs\paragraph_881.txt with 12 words!

Writing longest chain for data/paragraphs\paragraph_882.txt with 14 words!

Writing longest chain for data/paragraphs\paragraph_883.txt with 3 words!

Writing longest chain for data/paragraphs\paragraph_884.txt with 15 words!

Writing longest chain for data/paragraphs\paragraph_885.txt with 6 words!

Writing longest chai

Writing longest chain for data/paragraphs\paragraph_983.txt with 4 words!

Writing longest chain for data/paragraphs\paragraph_984.txt with 10 words!

Writing longest chain for data/paragraphs\paragraph_986.txt with 20 words!

Writing longest chain for data/paragraphs\paragraph_987.txt with 4 words!

Writing longest chain for data/paragraphs\paragraph_988.txt with 4 words!

Writing longest chain for data/paragraphs\paragraph_989.txt with 2 words!

Writing longest chain for data/paragraphs\paragraph_99.txt with 6 words!

Writing longest chain for data/paragraphs\paragraph_990.txt with 4 words!

Writing longest chain for data/paragraphs\paragraph_992.txt with 3 words!

Writing longest chain for data/paragraphs\paragraph_993.txt with 6 words!

Writing longest chain for data/paragraphs\paragraph_994.txt with 5 words!

Writing longest chain for data/paragraphs\paragraph_995.txt with 2 words!

Writing longest chain for data/paragraphs\paragraph_996.txt with 3 words!

Writing longest chain fo

--------

In [169]:
genre = 'conversation'

with open('results/LC_{}.txt'.format(genre), 'w') as f:
    train_set = pd.read_csv('data/train/{}/words.csv'.format(genre), encoding='ISO--8859-1', na_filter=False)
    X_train, test_fold_ids, vocabulary, _, y_train = preprocess_data(train_set, want_lc=True)

    unigram_classifier = UnigramClassifier(
        verbose=10, random_state=0, C=10, penalty='l1',
        solver='liblinear', class_weight='balanced', max_iter=500,
        vocabulary=vocabulary
    )

    C = np.logspace(-2,4,20)
    penalty = ['l1', 'l2']
    hyperparameters = {'C':C, 'penalty':penalty}
    clf = GridSearchCV(unigram_classifier, hyperparameters,
                       cv=PredefinedSplit(test_fold_ids),
                       verbose=10, scoring='f1', n_jobs=-1)

    clf.fit(X_train, y_train)

    test_set = pd.read_csv('data/test/{}/words.csv'.format(genre), encoding='ISO--8859-1', na_filter=False)
    X_test, _, _, _, y_test = preprocess_data(test_set, vocabulary=vocabulary, want_lc=True)
    preds = clf.predict(X_test)

    result = classification_report(y_test, preds, output_dict=True)
    print(result)
    f.write(json.dumps(result))

conversation
------(15145, 2874)


0

C:\Users\ezutp\Anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


matrix([[0, 1, 1, ..., 0, 0, 0],
        [1, 0, 1, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]], dtype=int32)

(15145, 2875)
Fitting 10 folds for each of 40 candidates, totalling 400 fits

...

[LibLinear]conversation
--(2650, 2874)


0

C:\Users\ezutp\Anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 1, 0],
        ...,
        [0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

(2650, 2875)
{'False': {'precision': 0.9050337311883757, 'recall': 0.7364864864864865, 'f1-score': 0.8121071012805587, 'support': 2368}, 'True': {'precision': 0.13692946058091288, 'recall': 0.35106382978723405, 'f1-score': 0.19701492537313434, 'support': 282}, 'micro avg': {'precision': 0.6954716981132075, 'recall': 0.6954716981132075, 'f1-score': 0.6954716981132075, 'support': 2650}, 'macro avg': {'precision': 0.5209815958846443, 'recall': 0.5437751581368603, 'f1-score': 0.5045610133268466, 'support': 2650}, 'weighted avg': {'precision': 0.8232958427690155, 'recall': 0.6954716981132075, 'f1-score': 0.7466520093538064, 'support': 2650}}


In [164]:
X_test, _, _, _, y_test = preprocess_data(test_set, vocabulary=vocabulary, want_lc=True)
preds = clf.predict(X_test)
result = classification_report(y_test, preds, output_dict=True)
print(result)

academic
----(8626, 7328)


85

C:\Users\ezutp\Anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


matrix([[0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 1, 0],
        ...,
        [0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 1, 0]], dtype=int32)

(8626, 7329)
{'False': {'precision': 0.7738383024442653, 'recall': 0.42719454329774614, 'f1-score': 0.5504920225470527, 'support': 6744}, 'True': {'precision': 0.21211503161329798, 'recall': 0.5526036131774708, 'f1-score': 0.3065585851142225, 'support': 1882}, 'micro avg': {'precision': 0.4545559935079991, 'recall': 0.4545559935079991, 'f1-score': 0.4545559935079991, 'support': 8626}, 'macro avg': {'precision': 0.4929766670287816, 'recall': 0.4898990782376085, 'f1-score': 0.4285253038306376, 'support': 8626}, 'weighted avg': {'precision': 0.6512828658915316, 'recall': 0.4545559935079991, 'f1-score': 0.49727120997476115, 'support': 8626}}


In [78]:
preds = clf.predict(X_test)
result = classification_report(y_test, preds, output_dict=True)
result

{'False': {'precision': 0.8100574712643678,
  'recall': 0.7086475615887381,
  'f1-score': 0.7559667471171897,
  'support': 3978},
 'True': {'precision': 0.23143236074270557,
  'recall': 0.3455445544554455,
  'f1-score': 0.27720413026211277,
  'support': 1010},
 'micro avg': {'precision': 0.6351242983159583,
  'recall': 0.6351242983159583,
  'f1-score': 0.6351242983159583,
  'support': 4988},
 'macro avg': {'precision': 0.5207449160035367,
  'recall': 0.5270960580220918,
  'f1-score': 0.5165854386896512,
  'support': 4988},
 'weighted avg': {'precision': 0.6928940066238547,
  'recall': 0.6351242983159583,
  'f1-score': 0.6590240360057968,
  'support': 4988}}

## Evaluation

In [1]:
%run model_functions.ipynb

In [2]:
genre = 'fiction'
name = 'LC'
do_experiment(
    genre=genre,
    name=name,
    want_lc=True
)

fiction
-----------(17879, 6033)


290

C:\Users\ezutp\Anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


matrix([[0, 1, 1, ..., 0, 0, 0],
        [1, 0, 1, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0]], dtype=int32)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


...

[LibLinear]fiction
--(3361, 6033)


127

C:\Users\ezutp\Anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:71: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


matrix([[0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 1, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

(3361, 6034)
{'False': {'precision': 0.8521536670547147, 'recall': 0.7740571025731406, 'f1-score': 0.8112301440709272, 'support': 2837}, 'True': {'precision': 0.18239795918367346, 'recall': 0.2729007633587786, 'f1-score': 0.21865443425076453, 'support': 524}, 'micro avg': {'precision': 0.6959238321927997, 'recall': 0.6959238321927997, 'f1-score': 0.6959238321927997, 'support': 3361}, 'macro avg': {'precision': 0.5172758131191941, 'recall': 0.5234789329659596, 'f1-score': 0.5149422891608458, 'support': 3361}, 'weighted avg': {'precision': 0.7477347468153737, 'recall': 0.6959238321927997, 'f1-score': 0.7188440470921217, 'support': 3361}}
